<a href="https://colab.research.google.com/github/Leonard-CGC/google_colab/blob/Machine_Learning/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 上傳檔案

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving PlayTennis.csv to PlayTennis.csv
User uploaded file "PlayTennis.csv" with length 433 bytes


In [ ]:
import os
os.listdir()

['.config', 'PlayTennis.csv', 'sample_data']

# Decision Tree (ID3)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
D = pd.read_csv('PlayTennis.csv')
D.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [ ]:
y_label = ['Play Tennis']
X_label = D.columns.drop(y_label)
X = D[X_label]
y = D[y_label]

In [ ]:
class TreeNode:
  def __init__(self):
    self.label = None
    self.values = {}

class DecisionTree:
  def __init__(self):
    self.head = TreeNode()

  def predict(self, x):
    ptr = self.head
    while isinstance(ptr.values[x[ptr.label]], TreeNode):
      ptr = ptr.values[x[ptr.label]]
    return ptr.values[x[ptr.label]]

class DecisionTreeClassifier:
  def __init__(self):
    pass

  def fit(self, X, y) -> DecisionTree:
    if not isinstance(y, pd.Series):
      y = y[y.columns[0]]

    DT = DecisionTree()
    self.define_Node(DT.head, X, y)
    return DT
    
  def define_Node(self, ptr, X, y):
    maxGain = 0
    maxLabel = None
    for label in X.columns:
      gain = self.Information_Gain(X, y, label)
      if gain>maxGain:
        maxGain, maxLabel = gain, label
    ptr.label = maxLabel
    X_label = X.columns.drop(maxLabel)
    for value in set(X[maxLabel]):
      if self.Entropy(y[X[maxLabel]==value])==0:
        ptr.values[value] = y[X[maxLabel]==value].values[0]
      elif len(X_label)!=0:
        ptr.values[value] = TreeNode()
        self.define_Node(ptr.values[value], X[X[maxLabel]==value][X_label], y[X[maxLabel]==value])
      else:
        maxN = 0
        maxAnswer = None
        for answer in set(y):
          n = (y==answer).sum()
          if n>maxN:
            maxN, maxAnswer = n, answer
        ptr.values[value] = answer
  
  def Entropy(self, y):
    N = y.shape[0]
    entropy = 0
    for value in set(y):
      n = (y==value).sum()
      P = n/N
      entropy -= P*np.log2(P)
    return entropy
    
  def Information_Gain(self, X, y, label):
    N = X.shape[0]
    Gain = self.Entropy(y)
    for value in set(X[label]):
      n = (X[label]==value).sum()
      Gain -= n*self.Entropy(y[X[label]==value])/N
    return Gain

In [ ]:
cls = DecisionTreeClassifier()
DT = cls.fit(X,y)

In [ ]:
def preorder(ptr):
  if not isinstance(ptr, TreeNode):
    print(ptr)
    return
  print(ptr.label)
  for key in ptr.values:
    print(key)
    preorder(ptr.values[key])
    
preorder(DT.head)

Outlook
Rain
Wind
Strong
No
Weak
Yes
Overcast
Yes
Sunny
Humidity
Normal
Yes
High
No


In [ ]:
x = {'Outlook':'Rain', 'Temperature':'Hot', 'Humidity':'Normal', 'Wind':'Strong'}
DT.predict(x)

'No'